# Master Thesis Script
## Sentiment and emotion label validation with convenience sample

Using manual annotations of emotions and the neutrality label, done by 7 people, LEIA's labels are validated.

In [42]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import regex as re
import json
import matplotlib.pyplot as plt
import scipy
import statistics
import math
import random
import scipy.stats as stats
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [43]:
# load dataset with my annotations
df_ES = pd.read_csv("Data_Sample.csv", dtype = {"index":str})

In [44]:
df_ES_emotion = pd.read_csv("Data_Sample_Emotion_20230605.csv", dtype = {"index":str})

In [45]:
# load people's annotations
df_NH = pd.read_csv("Annotation_Task_Sentiment_Emotion_Nele.csv", dtype = {"index":str}, encoding="latin-1")

In [46]:
df_TB = pd.read_csv("Annotation_Task_Sentiment_Emotion_TB.csv", dtype = {"index":str}, encoding="latin-1")

In [47]:
df_OB = pd.read_csv("Annotation_Task_Sentiment_Emotion_OB.csv", dtype = {"index":str}, encoding="latin-1")

In [48]:
df_GU = pd.read_csv("Annotation_Task_Sentiment_Emotion_GU.csv", dtype = {"index":str}, encoding="latin-1")

In [49]:
df_MP = pd.read_csv("Annotation_Task_MP.csv", dtype = {"index":str}, encoding="latin-1")

In [50]:
df_BS = pd.read_csv("Annotation_Task_Sentiment_Emotion_BS.csv", dtype = {"index":str}, encoding="latin-1")

In [51]:
df_NH = df_NH.rename(columns= {"Neutral: True or False": "Neutral"})

In [52]:
df_TB = df_TB.rename(columns= {"Neutral: True or False": "Neutral","Emotion: anger, fear, happiness, sadness, affection": "Emotion"})

In [53]:
df_OB = df_OB.rename(columns= {"Neutral: True or False": "Neutral"})

In [54]:
df_BS["Emotion"] = df_BS["Emotion"].replace("Hapiness", "Happiness")

In [55]:
# Function to convert 'FALSE' to True and other values to False
def convert_to_bool(value):
    return value.upper() == 'FALSE'

# Assign boolean values in the 'col3' based on 'col2' values
df_OB['Neutral'] = df_OB['Emotion'].apply(convert_to_bool)
df_OB["Emotion"] = df_OB["Emotion"].replace('FALSE', np.nan)


In [56]:
df_NH = df_NH.rename(columns= {"Neutral": "Neutral_NH", "Emotion": "Emotion_NH",})
df_TB = df_TB.rename(columns= {"Neutral": "Neutral_TB", "Emotion": "Emotion_TB",})
df_OB = df_OB.rename(columns= {"Neutral": "Neutral_OB", "Emotion": "Emotion_OB",})
df_GU = df_GU.rename(columns= {"Neutral": "Neutral_GU", "Emotion": "Emotion_GU",})
df_MP = df_MP.rename(columns= {"Neutral": "Neutral_MP", "Emotion": "Emotion_MP",})
df_BS = df_BS.rename(columns= {"Neutral": "Neutral_BS", "Emotion": "Emotion_BS",})

In [57]:
merged_df = df_NH.merge(df_TB, on='index', how='outer').merge(df_OB, on='index', how='outer').merge(df_GU, on='index', how='outer').merge(df_MP, on='index', how='outer').merge(df_BS, on='index', how='outer')
merged_df["Emotion_MP"] = merged_df["Emotion_MP"].replace("Neutral", np.nan)

/tmp/ipykernel_54433/4011044013.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'text_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = df_NH.merge(df_TB, on='index', how='outer').merge(df_OB, on='index', how='outer').merge(df_GU, on='index', how='outer').merge(df_MP, on='index', how='outer').merge(df_BS, on='index', how='outer')
/tmp/ipykernel_54433/4011044013.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Unnamed: 0_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_df = df_NH.merge(df_TB, on='index', how='outer').merge(df_OB, on='index', how='outer').merge(df_GU, on='index', how='outer').merge(df_MP, on='index', how='outer').merge(df_BS, on='index', how='outer')
/tmp/ipykernel_54433/4011044013.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'text_x'} in the result is deprecated and will raise a MergeError in a future version.
  me

In [20]:
merged_df

,Column1,index,text_x,Neutral_NH,Emotion_NH,Unnamed: 0_x,text_y,Neutral_TB,Emotion_TB,Unnamed: 0_y,...,Unnamed: 0_y,text_x,Unnamed: 3,Neutral_MP,Emotion_MP,TRUE,FALSE,Neutral_BS,Emotion_BS,text_y
0,0,241309,Neighbors helping neighbors. That's what our c...,False,Happiness,0,Neighbors helping neighbors. That's what our c...,False,Affection,0,...,0,Neighbors helping neighbors. That's what our c...,NaN,False,Affection,NaN,X,False,Happiness,Neighbors helping neighbors. That's what our c...
1,1,128570,The drugs that don't kill Americans are making...,True,NaN,1,The drugs that don't kill Americans are making...,False,Fear,1,...,1,The drugs that don't kill Americans are making...,NaN,False,Anger,NaN,X,False,Anger,The drugs that don't kill Americans are making...
2,2,6010,We've launched the #GA08 2018 Congressional Ap...,True,NaN,2,We've launched the #GA08 2018 Congressional Ap...,True,NaN,2,...,2,We've launched the #GA08 2018 Congressional Ap...,NaN,True,NaN,X,NaN,True,NaN,We've launched the #GA08 2018 Congressional Ap...
3,3,508998,As we continue to reckon with police violence ...,True,NaN,3,As we continue to reckon with police violence ...,True,NaN,3,...,"""3","""""As we continue to reckon with police violenc...",NaN,False,Sadness,X,NaN,True,NaN,As we continue to reckon with police violence ...
4,4,311558,.@BristolCC's future National Offshore Wind In...,True,NaN,4,.@BristolCC's future National Offshore Wind In...,True,NaN,4,...,4,.@BristolCC's future National Offshore Wind In...,NaN,False,Affection,X,NaN,True,NaN,.@BristolCC's future National Offshore Wind In...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,196,197226,"Since the #Steelers didn't make it this year, ...",True,NaN,196,"Since the #Steelers didn't make it this year, ...",True,NaN,196,...,"""196","""""Since the #Steelers didn't make it this year...",NaN,False,Affection,X,NaN,True,NaN,"Since the #Steelers didn't make it this year, ..."
197,197,150357,Let's not risk the health of our entire next g...,True,NaN,197,Let's not risk the health of our entire next g...,False,Fear,197,...,"""197","""""Let's not risk the health of our entire next...",,False,Fear,NaN,X,False,Fear,Let's not risk the health of our entire next g...
198,198,126095,The new GDP numbers are out - and our economy ...,False,Happiness,198,The new GDP numbers are out - and our economy ...,False,Happiness,198,...,"""198","""""The new GDP numbers are out - and our econom...",NaN,False,Affection,NaN,X,False,Happiness,The new GDP numbers are out - and our economy ...
199,199,131226,Retaining cybersecurity experts in Augusta wou...,True,NaN,199,Retaining cybersecurity experts in Augusta wou...,False,Fear,199,...,199,Retaining cybersecurity experts in Augusta wou...,NaN,False,Affection,X,NaN,True,NaN,Retaining cybersecurity experts in Augusta wou...


In [18]:
print(len(df_NH), len(df_TB), len(df_OB), len(merged_df), len(df_BS))

201 201 201 201 201


In [58]:
merged_df = merged_df[["index", "text_x", "Neutral_NH", "Neutral_TB", "Neutral_OB", "Emotion_NH", "Emotion_TB", "Emotion_OB", "Neutral_GU", "Emotion_GU", "Neutral_MP", "Emotion_MP", "Emotion_BS", "Neutral_BS"]]

In [59]:
df_ES_emotion = df_ES_emotion[["index", "text", "Emotion_Esther", "Esther_Annotation", "Neutral_Sentiment", "Anger", "Fear", "Happiness", "Sadness", "Affection"]]

In [60]:
df_ES_emotion = df_ES_emotion.rename(columns = {"Esther_Annotation": "Neutral_ES", "Emotion_Esther":"Emotion_ES"})

In [61]:

df_ES_emotion['Neutral_ES'] = df_ES_emotion['Neutral_ES'].replace('Neutral', True).replace({"Positive": False}).replace({"Negative": False})


In [62]:
# now merge people's annotations with mine
merged_total_df = merged_df.merge(df_ES_emotion, on='index', how='left')

In [26]:
df_ES_emotion

,index,text,Emotion_ES,Neutral_ES,Neutral_Sentiment,Anger,Fear,Happiness,Sadness,Affection
0,241309,Neighbors helping neighbors. That’s what our c...,Affection,False,0.033531,0.052062,0.039879,0.140377,0.032218,0.735465
1,128570,The drugs that don’t kill Americans are making...,Fear,False,0.411985,0.312588,0.423851,0.029082,0.105790,0.128688
2,6010,We've launched the #GA08 2018 Congressional Ap...,Affection,True,0.333710,0.159957,0.095475,0.153080,0.030387,0.561101
3,508998,As we continue to reckon with police violence ...,Affection,False,0.133346,0.339769,0.075548,0.051793,0.162102,0.370787
4,311558,.@BristolCC's future National Offshore Wind In...,Happiness,False,0.272830,0.051205,0.038626,0.513876,0.032729,0.363564
...,...,...,...,...,...,...,...,...,...,...
210,169205,Don't ever let anyone tell you what @TheDemocr...,Affection,False,0.266531,0.228063,0.103401,0.157279,0.031130,0.480128
211,214922,Congrats to @CityofRacine for winning the Smar...,Happiness,False,0.014499,0.057121,0.030428,0.598988,0.033118,0.280345
212,479973,"I join millions of people in Israel, the US, &...",Sadness,False,0.518897,0.045373,0.038002,0.030843,0.755864,0.129918
213,211010,"On A-10 delay: ""This is the Admin's same ploy ...",Anger,False,0.238922,0.512299,0.304346,0.068334,0.042387,0.072634


In [127]:
# assign boolean Neutral value, according to the set threshold
def check_above_threshold(value):
    return value > 0.45

# Create a new column 'New_Column' based on the condition
merged_total_df['Neutral_Threshold'] = merged_total_df['Neutral_Sentiment'].apply(check_above_threshold)


In [63]:
merged_total_df["Neutral_ES"] = merged_total_df["Neutral_ES"].astype(bool)  

In [64]:
# now assign annotator neutral label, based on the label that is given most frequently to the tweet
# Function to check if the value appears at least twice and return boolean True/False
def check_two_true(value):
    return sum(value) > 3

# Create a new column 'New_Column' based on the condition
merged_total_df['Neutral_Annotators'] = merged_total_df[['Neutral_NH', 'Neutral_TB', 'Neutral_OB', "Neutral_GU", "Neutral_MP", "Neutral_BS"]].apply(check_two_true, axis=1)


In [65]:
merged_total_df.dtypes

index                  object
text_x                 object
text_x                 object
text_x                 object
Neutral_NH               bool
Neutral_TB               bool
Neutral_OB               bool
Emotion_NH             object
Emotion_TB             object
Emotion_OB             object
Neutral_GU               bool
Emotion_GU             object
Neutral_MP               bool
Emotion_MP             object
Emotion_BS             object
Neutral_BS               bool
text                   object
Emotion_ES             object
Neutral_ES               bool
Neutral_Sentiment     float64
Anger                 float64
Fear                  float64
Happiness             float64
Sadness               float64
Affection             float64
Neutral_Annotators       bool
dtype: object

In [66]:
pd.set_option('display.max_rows', None)

In [67]:
# Compute the F1 score
f1 = f1_score(merged_total_df['Neutral_Annotators'], merged_total_df['Neutral_ES'])


In [68]:
# neutral label F1 score
f1

0.7375886524822695

In [69]:
merged_total_df['Emotion_LEIA'] = merged_total_df[["Anger","Fear", "Happiness", "Sadness", "Affection"]].idxmax(axis=1)
# assign 6th label to this column
merged_total_df.loc[merged_total_df['Neutral_ES'], 'Emotion_LEIA'] = 'Neutral'

In [70]:
merged_total_df

,index,text_x,text_x,text_x,Neutral_NH,Neutral_TB,Neutral_OB,Emotion_NH,Emotion_TB,Emotion_OB,...,Emotion_ES,Neutral_ES,Neutral_Sentiment,Anger,Fear,Happiness,Sadness,Affection,Neutral_Annotators,Emotion_LEIA
0,241309,Neighbors helping neighbors. That's what our c...,Neighbors helping neighbors. That's what our c...,Neighbors helping neighbors. That's what our c...,False,False,False,Happiness,Affection,Affection,...,Affection,False,0.033531,0.052062,0.039879,0.140377,0.032218,0.735465,False,Affection
1,128570,The drugs that don't kill Americans are making...,The drugs that don't kill Americans are making...,The drugs that don't kill Americans are making...,True,False,False,NaN,Fear,Anger,...,Fear,False,0.411985,0.312588,0.423851,0.029082,0.105790,0.128688,False,Fear
2,6010,We've launched the #GA08 2018 Congressional Ap...,We've launched the #GA08 2018 Congressional Ap...,We've launched the #GA08 2018 Congressional Ap...,True,True,True,NaN,NaN,NaN,...,Affection,True,0.333710,0.159957,0.095475,0.153080,0.030387,0.561101,True,Neutral
3,508998,As we continue to reckon with police violence ...,As we continue to reckon with police violence ...,"""""As we continue to reckon with police violenc...",True,True,False,NaN,NaN,Anger,...,Affection,False,0.133346,0.339769,0.075548,0.051793,0.162102,0.370787,False,Affection
4,311558,.@BristolCC's future National Offshore Wind In...,.@BristolCC's future National Offshore Wind In...,.@BristolCC's future National Offshore Wind In...,True,True,True,NaN,NaN,NaN,...,Happiness,False,0.272830,0.051205,0.038626,0.513876,0.032729,0.363564,True,Happiness
5,195614,"Over the past year, our working mothers have s...","Over the past year, our working mothers have s...","""""Over the past year our working mothers have...",True,False,False,NaN,Anger,Affection,...,Affection,False,0.255290,0.109358,0.221575,0.023687,0.098643,0.546736,False,Affection
6,187945,Biden ignored the warning signs from intellige...,Biden ignored the warning signs from intellige...,"""""Biden ignored the warning signs from intelli...",False,False,False,Anger,Anger,Anger,...,Anger,False,0.078998,0.735628,0.115106,0.036766,0.075117,0.037383,False,Anger
7,288397,REMINDER: Applications are open for my Summer ...,REMINDER: Applications are open for my Summer ...,"""""REMINDER: Applications are open for my Summe...",True,True,True,NaN,NaN,NaN,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,Neutral
8,31335,"As Pope Francis has said, ""communion is not th...","As Pope Francis has said, ""communion is not th...","""""As Pope Francis has said """"""""communion is n...",True,False,False,NaN,Fear,Anger,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,True,Neutral
9,216785,NJ already gets less back in taxpayer money th...,NJ already gets less back in taxpayer money th...,"""""NJ already gets less back in taxpayer money ...",False,False,False,Anger,Affection,Affection,...,Anger,False,0.427901,0.436715,0.126041,0.053985,0.037640,0.345618,False,Anger


In [71]:
merged_total_df["Emotion_NH"] = merged_total_df["Emotion_NH"].replace(np.nan, "Neutral")
merged_total_df["Emotion_TB"] = merged_total_df["Emotion_TB"].replace(np.nan, "Neutral")
merged_total_df["Emotion_OB"] = merged_total_df["Emotion_OB"].replace(np.nan, "Neutral")
merged_total_df["Emotion_GU"] = merged_total_df["Emotion_GU"].replace(np.nan, "Neutral")
merged_total_df["Emotion_MP"] = merged_total_df["Emotion_MP"].replace(np.nan, "Neutral")
merged_total_df["Emotion_BS"] = merged_total_df["Emotion_BS"].replace(np.nan, "Neutral")
merged_total_df["Emotion_ES"] = merged_total_df["Emotion_ES"].replace(np.nan, "Neutral")

In [88]:
def get_value_occurs_twice(row, col1, col2, col3, col4, col5, col6, col7):
    selected_columns = [col1, col2, col3, col4, col5, col6, col7]
    unique_values = row[selected_columns].unique()
    for value in unique_values:
        if row[selected_columns].tolist().count(value) > 3:
            return value
    return "None"

# Select the names of the three columns you want to check
column1 = 'Emotion_NH'
column2 = 'Emotion_TB'
column3 = 'Emotion_OB'
column4 = 'Emotion_GU'
column5 = 'Emotion_MP'
column6 = 'Emotion_BS'
column7 = 'Emotion_ES'

# Create a new column 'new_column' based on the condition, using the selected columns
merged_total_df['Emotion_ALL'] = merged_total_df.apply(get_value_occurs_twice, axis=1, args=(column1, column2, column3, column4, column5, column6, column7))


In [82]:
merged_total_df["Emotion_ALL"].value_counts()

Neutral      71
Anger        32
Affection    20
Happiness    20
Fear         12
Sadness       6
Name: Emotion_ALL, dtype: int64

In [85]:
merged_total_df

,index,text_x,text_x,text_x,Neutral_NH,Neutral_TB,Neutral_OB,Emotion_NH,Emotion_TB,Emotion_OB,...,Neutral_ES,Neutral_Sentiment,Anger,Fear,Happiness,Sadness,Affection,Neutral_Annotators,Emotion_LEIA,Emotion_ALL
0,241309,Neighbors helping neighbors. That's what our c...,Neighbors helping neighbors. That's what our c...,Neighbors helping neighbors. That's what our c...,False,False,False,Happiness,Affection,Affection,...,False,0.033531,0.052062,0.039879,0.140377,0.032218,0.735465,False,Affection,Affection
1,128570,The drugs that don't kill Americans are making...,The drugs that don't kill Americans are making...,The drugs that don't kill Americans are making...,True,False,False,Neutral,Fear,Anger,...,False,0.411985,0.312588,0.423851,0.029082,0.105790,0.128688,False,Fear,Anger
2,6010,We've launched the #GA08 2018 Congressional Ap...,We've launched the #GA08 2018 Congressional Ap...,We've launched the #GA08 2018 Congressional Ap...,True,True,True,Neutral,Neutral,Neutral,...,True,0.333710,0.159957,0.095475,0.153080,0.030387,0.561101,True,Neutral,Neutral
3,508998,As we continue to reckon with police violence ...,As we continue to reckon with police violence ...,"""""As we continue to reckon with police violenc...",True,True,False,Neutral,Neutral,Anger,...,False,0.133346,0.339769,0.075548,0.051793,0.162102,0.370787,False,Affection,None
4,311558,.@BristolCC's future National Offshore Wind In...,.@BristolCC's future National Offshore Wind In...,.@BristolCC's future National Offshore Wind In...,True,True,True,Neutral,Neutral,Neutral,...,False,0.272830,0.051205,0.038626,0.513876,0.032729,0.363564,True,Happiness,Neutral
5,195614,"Over the past year, our working mothers have s...","Over the past year, our working mothers have s...","""""Over the past year our working mothers have...",True,False,False,Neutral,Anger,Affection,...,False,0.255290,0.109358,0.221575,0.023687,0.098643,0.546736,False,Affection,None
6,187945,Biden ignored the warning signs from intellige...,Biden ignored the warning signs from intellige...,"""""Biden ignored the warning signs from intelli...",False,False,False,Anger,Anger,Anger,...,False,0.078998,0.735628,0.115106,0.036766,0.075117,0.037383,False,Anger,Anger
7,288397,REMINDER: Applications are open for my Summer ...,REMINDER: Applications are open for my Summer ...,"""""REMINDER: Applications are open for my Summe...",True,True,True,Neutral,Neutral,Neutral,...,True,NaN,NaN,NaN,NaN,NaN,NaN,True,Neutral,Neutral
8,31335,"As Pope Francis has said, ""communion is not th...","As Pope Francis has said, ""communion is not th...","""""As Pope Francis has said """"""""communion is n...",True,False,False,Neutral,Fear,Anger,...,True,NaN,NaN,NaN,NaN,NaN,NaN,True,Neutral,Neutral
9,216785,NJ already gets less back in taxpayer money th...,NJ already gets less back in taxpayer money th...,"""""NJ already gets less back in taxpayer money ...",False,False,False,Anger,Affection,Affection,...,False,0.427901,0.436715,0.126041,0.053985,0.037640,0.345618,False,Anger,Anger


In [300]:
# compute F1 score between Emotion_LEIA with each annotators emotion column, take average
cr_NH = classification_report(merged_total_df['Emotion_NH'], merged_total_df['Emotion_LEIA'])
cr_TB = classification_report(merged_total_df['Emotion_TB'], merged_total_df['Emotion_LEIA'])
cr_OB = classification_report(merged_total_df['Emotion_OB'], merged_total_df['Emotion_LEIA'])
cr_GU = classification_report(merged_total_df['Emotion_GU'], merged_total_df['Emotion_LEIA'])
cr_MP = classification_report(merged_total_df['Emotion_MP'], merged_total_df['Emotion_LEIA'])

In [301]:
print(cr_NH)

              precision    recall  f1-score   support

   Affection       0.26      0.83      0.39        12
       Anger       0.41      0.94      0.58        18
        Fear       0.27      0.67      0.38         6
   Happiness       0.50      0.62      0.55        21
     Neutral       0.97      0.57      0.72       130
     Sadness       0.75      0.21      0.33        14

    accuracy                           0.60       201
   macro avg       0.53      0.64      0.49       201
weighted avg       0.79      0.60      0.63       201



In [302]:
print(cr_TB)

              precision    recall  f1-score   support

   Affection       0.49      0.68      0.57        28
       Anger       0.54      0.56      0.55        39
        Fear       0.47      0.26      0.33        27
   Happiness       0.46      0.63      0.53        19
     Neutral       0.72      0.70      0.71        79
     Sadness       0.75      0.33      0.46         9

    accuracy                           0.59       201
   macro avg       0.57      0.53      0.53       201
weighted avg       0.60      0.59      0.58       201



In [303]:
print(cr_OB)

              precision    recall  f1-score   support

   Affection       0.21      0.53      0.30        15
       Anger       0.59      0.56      0.57        43
        Fear       0.33      0.36      0.34        14
   Happiness       0.58      0.62      0.60        24
     Neutral       0.72      0.61      0.66        90
     Sadness       1.00      0.27      0.42        15

    accuracy                           0.55       201
   macro avg       0.57      0.49      0.48       201
weighted avg       0.63      0.55      0.57       201



In [304]:
print(cr_GU)

              precision    recall  f1-score   support

   Affection       0.46      0.67      0.55        27
       Anger       0.61      0.58      0.60        43
        Fear       0.47      0.39      0.42        18
   Happiness       0.77      0.45      0.57        44
     Neutral       0.54      0.89      0.67        46
     Sadness       1.00      0.17      0.30        23

    accuracy                           0.57       201
   macro avg       0.64      0.53      0.52       201
weighted avg       0.64      0.57      0.55       201



In [305]:
print(cr_MP)

              precision    recall  f1-score   support

   Affection       0.49      0.44      0.46        43
       Anger       0.59      0.60      0.59        40
        Fear       0.53      0.40      0.46        20
   Happiness       0.46      0.57      0.51        21
     Neutral       0.61      0.85      0.71        54
     Sadness       1.00      0.17      0.30        23

    accuracy                           0.56       201
   macro avg       0.61      0.51      0.50       201
weighted avg       0.60      0.56      0.54       201



In [184]:
# F1 score anger average
avg_anger_F1 = (0.58+0.55+0.57)/3
print(avg_anger_F1)

0.5666666666666665


In [185]:
# F1 score happiness average
avg_hap_F1 = (0.55+0.53+0.60)/3
print(avg_hap_F1)

0.56


In [186]:
# F1 score fear average
avg_fear_F1 = (0.38+0.33+0.34)/3
print(avg_fear_F1)

0.35000000000000003


In [187]:
# F1 score sadness average
avg_sad_F1 = (0.33+0.46+0.42)/3
print(avg_sad_F1)

0.4033333333333333


In [188]:
# F1 score affection average
avg_aff_F1 = (0.57+0.3+0.3)/3
print(avg_aff_F1)

0.38999999999999996


In [89]:
# compare with result that only takes high inter annotator agreement
# first drop observations where we have None, i.e., no high enough inter-annotator agreement
df_filtered = merged_total_df.dropna(subset=['Emotion_ALL'])
print(len(df_filtered), len(merged_total_df))
cr_ALL = classification_report(merged_total_df['Emotion_ALL'], merged_total_df['Emotion_LEIA'])

201 201


/usr/local/jupyterhub/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/jupyterhub/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/jupyterhub/lib64/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
print(cr_ALL)

              precision    recall  f1-score   support

   Affection       0.46      0.90      0.61        20
       Anger       0.61      0.78      0.68        32
        Fear       0.40      0.50      0.44        12
   Happiness       0.54      0.70      0.61        20
     Neutral       0.76      0.82      0.79        71
        None       0.00      0.00      0.00        40
     Sadness       0.75      0.50      0.60         6

    accuracy                           0.62       201
   macro avg       0.50      0.60      0.53       201
weighted avg       0.51      0.62      0.55       201



In [96]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
conf_matrix = multilabel_confusion_matrix(merged_total_df['Emotion_ALL'], merged_total_df['Emotion_LEIA'],
                                         labels=["Affection", "Anger", "Fear", "Happiness", "Sadness", "Neutral"])

print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[[160  21]
  [  2  18]]

 [[153  16]
  [  7  25]]

 [[180   9]
  [  6   6]]

 [[169  12]
  [  6  14]]

 [[194   1]
  [  3   3]]

 [[112  18]
  [ 13  58]]]
